In [3]:
!pip install spacy_transformers
!pip install -U spacy

In [26]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [3]:
spacy.__version__

'3.7.4'

In [4]:
!nvidia-smi

Sun Mar 10 09:55:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [6]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json', 'r'))

In [7]:
len(cv_data)

200

In [8]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text,annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity==True:
        continue

      entity_indices = entity_indices + list(range(start, end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode= 'strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [10]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size = 0.3)

In [11]:
len(train)

140

In [12]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 60/60 [00:00<00:00, 81.35it/s]


In [13]:
len(db.tokens)

60

In [29]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        5367.85   1213.60    0.17    0.10    0.51    0.00
  4     200      491835.84  90825.42   11.14   57.69    6.17    0.11
  8     400       90440.60  35798.63   41.10   46.21   37.00    0.41
 12     600        5610.32  29316.85   54.99   57.22   52.93    0.55
 

### Model Test

In [30]:
nlp = spacy.load('/content/output/model-best')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [32]:
doc = nlp('My name is Carlos Anton. I worked at VASS consultoría de sistemas S.L. I have 10 years of experience')

for ent in doc.ents:
  print(ent.text, "     ->>>>  ", ent.label_)

Carlos Anton      ->>>>   Name
10 years      ->>>>   Years of Experience


In [18]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.6 MB/s eta 0:00:00


In [9]:
!pip install fitz

In [11]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 36.4 MB/s eta 0:00:00


In [33]:
import sys, fitz  # fitz es un alias común para PyMuPDF
fname = '/content/CV-Parsing-using-Spacy-3/data/test/10228751.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    # Usa get_text() en lugar de getText()
    text = text + str(page.get_text())

# Limpia el texto eliminando saltos de línea innecesarios
tx = " ".join(text.split('\n'))
print(tx)

BUSINESS DEVELOPMENT MANAGER/STAFFING MANAGER Professional Summary Innovative Manager seeks position offering opportunities for new professional and personal challenges. Self-starter with a positive, can-do attitude who is driven to learn, improve and succeed. Over 10 years of proactive and indirect diverse recruiting and staffing experience. Education and Training 2003 Bachelor of Science : Health Science option in Health Management and Marketing California State University of Hayward ï¼​ City , State , United States Skill Highlights Staffing management ability Proven patience and self-discipline Relationship and team building Staff training and development Critical thinking proficiency Compensation/benefits administration Skilled negotiator Account management Excellent written and verbal communicator Enthusiastic team player Problem solving Billing Attention to detail Recruiting and selection techniques Proficient communicator Contract review Cold calling Hiring recommendations Inter

In [37]:
doc = nlp(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

DESIGNATION                   - BUSINESS DEVELOPMENT MANAGER/STAFFING MANAGER
GRADUATION YEAR               - 2003
DEGREE                        - Bachelor of Science : Health Science option in Health Management and Marketing
COLLEGE NAME                  - California State University of Hayward


In [35]:
from PyPDF2 import PdfReader

def clean_and_merge_text(text):
    """Limpia el texto y lo une para la extracción de entidades."""
    words = text.strip().split()
    clean_text = ' '.join(words)
    return clean_text

# Ruta al archivo PDF
file_path = '/content/CV-Parsing-using-Spacy-3/data/test/Alice Clark CV.pdf'

# Abre el archivo PDF y lee el contenido página por página
try:
    pdf_reader = PdfReader(file_path)
    for page_number, page in enumerate(pdf_reader.pages, start=1):
        page_text = page.extract_text()  # Extrae el texto de la página
        if page_text:  # Verifica si se extrajo texto
            clean_text = clean_and_merge_text(page_text)  # Limpia y une el texto de la página
            print(f"Texto limpio de la página {page_number}: {clean_text}\n---\n")
except Exception as e:
    print(f"Error al leer el archivo: {e}")

Texto limpio de la página 1: Alice Clark AI / Machine Learning Delhi, India Em ail me on Indeed • 20+ y ears of experience in data han dling, design , and development • Data Wa rehou se: Data analysis , star /snow fla ke scema data modelling and design specific to data warehousing and business intelligence • Database: Experience in database designing, scalability, back -up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U -SQL) Willing to relocate anywhere WORK EXPERIE NCE Software Engineer Microsoft – Bangalore, Karnataka January 2000 to Present 1. Microsoft Rewards Live dashboards: Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping online. Microsoft Rewards members can earn points when searchin

In [36]:
doc = nlp(clean_text)

for ent in doc.ents:
  print(ent.text, "     ->>>>  ", ent.label_)

### Descargar archivos Colab

In [21]:
from google.colab import files

In [38]:
!zip -r /content/file.zip /content/output

updating: content/output/ (stored 0%)
updating: content/output/model-last/ (stored 0%)
updating: content/output/model-last/meta.json (deflated 65%)
updating: content/output/model-last/ner/ (stored 0%)
updating: content/output/model-last/ner/cfg (deflated 33%)
updating: content/output/model-last/ner/moves (deflated 74%)
updating: content/output/model-last/ner/model (deflated 8%)
updating: content/output/model-last/config.cfg (deflated 61%)
updating: content/output/model-last/transformer/ (stored 0%)
updating: content/output/model-last/transformer/cfg (stored 0%)
updating: content/output/model-last/transformer/model (deflated 13%)
updating: content/output/model-last/tokenizer (deflated 81%)
updating: content/output/model-last/vocab/ (stored 0%)
updating: content/output/model-last/vocab/vectors (deflated 45%)
updating: content/output/model-last/vocab/strings.json (deflated 76%)
updating: content/output/model-last/vocab/key2row (stored 0%)
updating: content/output/model-last/vocab/lookups.

In [39]:
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>